# GROMACS CMDs

## tpr

In [7]:
import mdtraj as md
import nglview as nv

molecule="1fme"
molecule_lower = molecule.lower()
molecule_upper = molecule.upper()

In [8]:
print("\nCreate .gro file")
print(f"gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm27 -water tip3p --ignh")

print("\nCreate .tpr file")
print(f"gmx grompp -f {molecule_lower}.mdp -po {molecule_lower}.mdp -c folded.gro -p topol.top -o nvt_0.tpr -maxwarn 2")


Create .gro file
gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm27 -water tip3p --ignh

Create .tpr file
gmx grompp -f 1fme.mdp -po 1fme.mdp -c folded.gro -p topol.top -o nvt_0.tpr -maxwarn 2


## Force field

In [ ]:
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/atomtypes.atp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/lipids.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/lipids.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.c.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.c.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.c.tdb

/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/atomtypes.atp
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.hdb
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.rtp
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.n.tdb
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.c.tdb

residuetypes.dat 

8: CHARMM27 all-atom force field (CHARM22 plus CMAP for proteins)

1: TIP3P   TIP 3-point, recommended

# Equilibrium

## Explicit TIP3P

In [ ]:
## Initial system ##
# gro, top file: folded.gro, topol.top
gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm27 -water tip3p --ignh

# Create box: folded_boxed.gro
gmx editconf -f folded.gro -o folded_boxed.gro -c -d 1.2 -bt cubic -box 4.7 4.7 4.7

# Solvate: folded_solv.gro, topol.top
gmx solvate -cp folded_boxed.gro -cs spc216.gro -o folded_solv.gro -p topol.top

# Add ions: ions.tpr
gmx grompp -f ../common/ions.mdp -po ions.mdp -c folded_solv.gro -p topol.top -o ions.tpr -maxwarn 1
# select SOL (13): topol.top, folded_solv_ions.gro
gmx genion -s ions.tpr -o folded_solv_ions.gro -p topol.top -pname NA -nname CL -neutral



## Energy minimization ##
# Energy minimization: em.tpr
gmx grompp -f ../common/minim.mdp -c folded_solv_ions.gro -p topol.top -o em.tpr

# Energy minimization: em.log, em.edr, em.trr, em.gro
# em.log: ASCII-text log file of the EM process
# em.edr: Binary energy file
# em.trr: Binary full-precision trajectory
# em.gro: Energy-minimized structure
gmx mdrun -v -deffnm em
# Energy analysis (11, 0): potential.xvg
gmx energy -f em.edr -o potential.xvg


## Equilibration - temperature ##
# Equilibration - temperature: nvt.tpr, nvt.gro, nvt.cpt, nvt.edr, nvt.log, nvt.trr, nvt.log, nvt.xtc
# gmx grompp -f ../common/1fme_nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr
gmx grompp -f ../common/gtt_nvt.mdp -c em.gro -r em.gro -p topol.top -o nvt.tpr

gmx mdrun -deffnm nvt -ntomp 1 -pin on -bonded gpu -nb gpu -pme gpu

# Analysis - 16 0
gmx energy -f nvt.edr -o temperature.xvg

## Equilibration - pressure ##
# Equilibration - pressure : 
# gmx grompp -f ../common/1fme_npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr
gmx grompp -f ../common/gtt_npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p topol.top -o npt.tpr

gmx mdrun -deffnm npt -ntomp 1 -pin on -bonded gpu -nb gpu -pme gpu

# Analysis 
# 17 0
gmx energy -f npt.edr -o pressure.xvg 
# 23 0
gmx energy -f npt.edr -o density.xvg 



## Production MD ##
# Final tpr
# gmx grompp -f 1fme_md.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr
gmx grompp -f gtt_md.mdp -c npt.gro -t npt.cpt -p topol.top -o md.tpr

# Run MD
# gmx mdrun -deffnm md -ntomp 1


In [34]:
molecule="1fme"
gro_path = f"./{molecule.upper()}/folded_solv_ions.gro"
gro_traj = md.load(gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget()

In [38]:
molecule="1fme"
xtc_path = f"./{molecule.upper()}/nvt.xtc"
gro_path = f"./{molecule.upper()}/nvt.gro"
gro_traj = md.load(xtc_path, top=gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget(max_frame=50)

In [40]:
molecule="1fme"
gro_path = f"./{molecule.upper()}/npt.gro"
gro_traj = md.load(gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget()

In [ ]:
molecule="1fme"
gro_path = f"./{molecule.upper()}/folded.gro"
gro_traj = md.load(gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget()

In [43]:
molecule="1fme"
xtc_path = f"/home/shpark/prj-mlcv/lib/bioemu/opes/simulations/1fme/tica/debug/0.xtc"
gro_path = f"./{molecule.upper()}/folded.gro"
gro_traj = md.load_xtc(xtc_path, top=gro_path)
gro_traj.center_coordinates()
gro_view = nv.show_mdtraj(gro_traj)
gro_view.clear_representations()
gro_view.add_cartoon(selection="protein")
gro_view.add_ball_and_stick(selection="resname")
gro_view.add_ball_and_stick(selection="CL", radius=0.5)

gro_view

NGLWidget(max_frame=20)

## Implicit TIP3P

In [ ]:
gmx pdb2gmx -f folded.pdb -o folded.gro -p topol.top -ff charmm27 -water tip3p -ignh

# EM
gmx grompp -f em_gb.mdp -c gb_processed.gro -p topol.top -o em_gb.tpr
gmx mdrun  -deffnm em_gb

# NVT - restrained
gmx grompp -f nvt_gb_posres.mdp -c em_gb.gro -p topol.top -o nvt_gb_posres.tpr
gmx mdrun  -deffnm nvt_gb_posres

# NVT - unrestrained
gmx grompp -f nvt_gb.mdp -c nvt_gb_posres.gro -t nvt_gb_posres.cpt -p topol.top -o nvt_gb.tpr
gmx mdrun  -deffnm nvt_gb

# start OPES (GBSA)
gmx grompp -f md_opes_gb.mdp -c nvt_gb.gro -t nvt_gb.cpt -p topol.top -o opes_gb.tpr
gmx mdrun  -deffnm opes_gb -plumed plumed.dat